## Malthus



Among all the things ever written by social scientists (or
&ldquo;political philosophers&rdquo;, to use a term he would have recognized),
Malthus&rsquo; [&ldquo;Essay on Population&rdquo; (1798)](https://oll.libertyfund.org/titles/malthus-an-essay-on-the-principle-of-population-1798-1st-ed) must be among the most
influential.  You can find the first edition at the link above.
The main argument is spelled out in Chapter 1, which I encourage
you to read.  A key point of the essay begins with [two postulata](https://oll.libertyfund.org/titles/311#Malthus_0195_20):

-   **First:** That food is necessary to the existence of man.
-   **Secondly:** That the passion between the sexes is necessary and
    will remain nearly in its present state.

   These lead to a conclusion:
>    &ldquo;Assuming then, my postulata as granted, I say, that the power of
>    population is indefinitely greater than the power in the earth to
>    produce subsistence for man.  Population, when unchecked, increases
>    in a geometrical ratio. Subsistence increases only in an
>    arithmetical ratio. A slight acquaintance with numbers will shew
>    the immensity of the first power in comparison of the second.&rdquo;



### Malthus meets math



Malthus&rsquo; argument boils down to a mathematical assertion: that
population grows at an exponential rate, while food production
increases only linearly.  

If we lack a &ldquo;slight acquaintance&rdquo; with numbers a graph makes the
point:



In [1]:
from plotly.offline import init_notebook_mode, iplot
import numpy as np
from scipy.optimize import newton

init_notebook_mode(connected=True)

# Play with these constants
## for exponential
A = 1
r = 0.1

## for line
b = 10
m = 100

food_production = lambda t: b+m*t

population = lambda t: A*np.exp(r*t)

diff = lambda t: food_production(t) - population(t)

# Point where curves cross
# Point where curves cross
x0 = 10
tstar = -1
while tstar <= 0:
  tstar = newton(diff,x0,maxiter=1000)
  x0 = x0*10
    
print(tstar)

In [1]:
T=np.linspace(0,tstar*1.1,1000)


data = [dict(name='Food production',
             #visible = False,
             line=dict(color='blue', width=6),
             x = T,
             y = [food_production(t) for t in T]),
        dict(name='Population',
             #visible = False,
             line=dict(color='red', width=6),
             x = T,
             y = [population(t) for t in T])]

layout = dict()
fig = dict(data=data, layout=layout)

iplot(fig)

## Looking for food



What’s happened to food production over recent decades?



### Preface



In [1]:
#!import wbdata
#!import cufflinks

import wbdata
import cufflinks as cf
cf.go_offline()

def fix_date_index(df):
    idx_vars = df.index.names
    new = df.reset_index()
    new.date = new.date.astype(int)
    return new.set_index(idx_vars)

### Finding data



Maybe there&rsquo;s something relevant to food production in the WDI
indicators?



In [1]:
wbdata.get_topic()

&ldquo;Agriculture & Rural Development&rdquo; sounds relevant.  Let&rsquo;s see what
variables are available:



In [1]:
wbdata.get_indicator(topic=1)

### Cereal production & land



The great majority of food (as measured by calories) originates in the
form of cereals; some of these are turned into products directly
consumed by humans (bread, rice, hominy); others are used to feed
livestock, which humans subsequently eat.  So what&rsquo;s happened to
cereal production over the last few decades?



In [1]:
vars = {"AG.PRD.CREL.MT":"Cereal production"}

# Use this to find top producers:
#big_producers = cereals.query('date==2000').squeeze().sort_values(ascending=False)
#big_producers.head(20)

use=["CHN","USA","IND","FRA","RUS","WLD"]

cereals = fix_date_index(wbdata.get_dataframe(vars,country=use)).squeeze().unstack('country')

cereals.iplot(xTitle="Year",yTitle="Metric Tons",
              title="Cereal Production by Top 5 (in 2000)")

### FAO Food Index



The FAO compiles a series of data meant to measure changes in the
   supply of food of all sorts, including both plant and animal
   sources.  Let&rsquo;s how that&rsquo;s changed over recent decades:



In [1]:
vars = {"AG.PRD.LVSK.XD":"Livestock index",
        "AG.PRD.FOOD.XD":"Food index",
        "AG.PRD.CROP.XD":"Crop index"}

food = fix_date_index(wbdata.get_dataframe(vars,country='WLD'))

food.iplot(xTitle="Year",yTitle="Index")

From this we don&rsquo;t get levels; the indices are normalized so that 2006
is 100.  Let&rsquo;s compare this with a similarly normalized description of
world population:



In [1]:
vars = {"AG.PRD.LVSK.XD":"Livestock index",
        "AG.PRD.FOOD.XD":"Food index",
        "AG.PRD.CROP.XD":"Crop index",
        "SP.POP.TOTL":"Population"}

food = fix_date_index(wbdata.get_dataframe(vars,country='WLD'))

food.sort_index(inplace=True)

food['Population'] = food['Population']/food.loc[2006,'Population']*100

food.iplot(xTitle="Year",yTitle="Index")

And here in growth rates:



In [1]:
np.log(food).diff().iplot(xTitle="Year",yTitle="Growth rates")